In [1]:
import pandas as pd   #IMPORTING PANDAS
import numpy as np   #IMPORTING NUMPY

In [2]:
file="C:/Users/VenkatBhaskar/Downloads/proj1_testsignal2.unknown"   #REFERENCING FILE 

In [3]:
df=pd.read_table(file,delimiter='\n',header=None)   #LOADING INTO DATAFRAME

In [18]:
df

,0
0,564.3873
1,748.8244
2,457.1925
3,515.4503
4,106.0660
...,...
2999995,439.0900
2999996,227.7060
2999997,380.7571
2999998,181.2981


In [6]:
df.shape

(3000000, 1)

In [14]:
preambleLengthInSamples = 800  #8 SYMBOLS IS 800 SAMPLES
firstSymbol=0
#firstSymbol= 1014939  #GIVEN
lastSymbol= 2999999   #GIVEN FILE
symbolLength = 100    #1 SYMBOL= 100 SAMPLES
currentPosition= 0
byteStream = []       #FOR STORING 1 BIT PER 100 SAMPLES

In [15]:
sampleMean= int(df[0:9999].mean())     #MEAN OF FIRST 10K
sampleStandardDeviation= int(df[0:9999].std())   #STANDARD DEVITION OF FIRST 10K
sampleThreshold = sampleMean*4 + sampleStandardDeviation*8    #THRESHOLD CALCULATION USING FORMULA GIVEN

print('Mean: ', sampleMean)
print('Standard Deviation: ', sampleStandardDeviation)
print('Threshold: ', sampleThreshold)
print('Length of preamble in samples: ', preambleLengthInSamples)
#print('Threshold break: ',firstSymbol)

Mean:  486
Standard Deviation:  217
Threshold:  3680
Length of preamble in samples:  800


In [20]:
for i,r in df.iterrows():
    if df.loc[i,0]>sampleThreshold:
        firstSymbol=i
        break
print(firstSymbol)

1045366


In [21]:
df_symbol = df[firstSymbol + preambleLengthInSamples -1:]   #INDEXING ORIGINAL FILE/ DATAFRAME TO EXCLUDE PREAMBLE AND SAMPLES BEFORE THRESHOLD BREACH

sampleList = df[firstSymbol + preambleLengthInSamples -1:].values.tolist() #FORMATTING DATA FRAME TO LIST FOR CONVINIENCE
#print(firstSymbol + preambleLengthInSamples -1)
sList = []   #EMPTY LIST TO STORE ALL SAMPLES POST PREAMBLE IN A LIST

for i in range(0,len(sampleList),1):
  sList.append(sampleList[i][0])

print("First value in Sample List- sList: ", sList[0])
print("Number of Samples left to process after first data symbol + preamble: ", len(sList))


First value in Sample List- sList:  2223.6495
Number of Samples left to process after first data symbol + preamble:  1953835


In [22]:
current=0  #CURRENT POSITION OF INDEX IN sList
reshapedSamples=[] #SHAPING LIST SUCH THAT shape(a)= 100X19483
for i in range(0,len(sList),100):
    reshapedSamples.append(sList[current:current+100])
    current+=100

In [23]:
print("The 100 amplitudes of first symbol are: ", reshapedSamples[1])

The 100 amplitudes of first symbol are:  [739.8763, 719.1217, 941.9114, 648.896, 665.1955, 842.3064, 733.8351, 706.1253, 614.9943, 867.4388, 941.0388, 482.9016, 673.9177, 614.7422, 634.0, 612.9152, 619.122, 631.229, 849.6599, 659.7348, 496.677, 767.9095, 570.5927, 678.7533, 965.6097, 897.8775, 735.1313, 589.6185, 601.247, 706.7227, 824.4325, 622.8523, 525.9743, 765.763, 751.5118, 549.0, 512.2695, 779.4164, 708.7433, 809.9636, 669.0747, 656.9117, 637.8409, 588.3349, 635.4982, 598.1613, 663.23, 568.3775, 753.5682, 1899.1274, 3392.0001, 5332.2045, 6503.3876, 6645.2493, 6253.0541, 4759.8992, 2642.9355, 1177.3806, 617.4374, 691.755, 553.8059, 623.5415, 923.9897, 238.6064, 442.6206, 610.404, 668.1392, 496.1794, 430.7435, 408.0159, 467.6719, 311.8108, 525.3427, 455.728, 690.8871, 589.9941, 313.6399, 310.9421, 500.6396, 284.9719, 358.1983, 367.0422, 535.6053, 632.1139, 221.0543, 103.5857, 609.6425, 253.1265, 374.2259, 351.693, 590.2076, 318.4776, 141.9472, 254.1692, 262.185, 252.121, 350.1428,

In [24]:
print("The reshaped sample list contains",len(reshapedSamples[0]), "x", len(reshapedSamples), "amplitudes")

The reshaped sample list contains 100 x 19539 amplitudes


In [25]:
bitStream=[]   #ACTUAL BITSTREAM TO STORE THRESHOLD BREACH SCENARIOS OF EACH 100 SAMPLES OR 1 SYMBOL

def demodulator(bStream):
    for i in reshapedSamples:
        for j in range(len(i)):
            if j<10 and i[j]>sampleThreshold:
                bitStream.append(0)
                break
            elif j>53 and j<63 and i[j]>sampleThreshold:
                bitStream.append(1)
                break
            elif j==99:
                bitStream.append(0)

                
demodulator(bitStream)

print("Number of bits: ", len(bitStream))

Number of bits:  19538


In [26]:
print("Demodulated Bit Stream: ",bitStream)

Demodulated Bit Stream:  [0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,

In [27]:
byteStream=[]   #BIT STREAM FOR STORING LEFT 4 BITS IN 7 SYMBOLS
currentPos=0
for i in range(0,len(bitStream),7):
    byteStream.append(bitStream[currentPos:currentPos+4])   #APPENDING ONLY FIRST 4 BITS OF EVERY 7 BITS
    currentPos+=7

In [28]:
byteS=[]   #X CONTAINS ALL THE ABOVE FORMATTED 4 BITS AS A LIST WITH 11340 BITS IN IT
for i in byteStream:
    for j in i:
        byteS.append(j) 

In [29]:
print("Total bytes: ", len(byteS))

print("Bytes in List format: ",byteS)

Total bytes:  11165
Bytes in List format:  [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,

In [32]:
byteToStr=''.join(str(i) for i in byteS)   #CONVERTING LIST TO STRING

In [33]:
# print(type(byteToStr))
print("Final Byte Stream converted into a string: ",byteToStr)

Final Byte Stream converted into a string:  01001001011011100010000001100001001000000111010001111001011100000110100101100011011000010110110000100000011101110110100101110010011001010110110001100101011100110111001100100000011011100110010101110100011101110110111101110010011010110010110000100000011000010010000001100010011000010111001101100101001000000111001101110100011000010111010001101001011011110110111000100000011011110111001000100000011000010010000001110010011011110111010101110100011001010111001000100000011011010110000101111001001000000110001001100101001000000110000101110011011100110110111101100011011010010110000101110100011001010110010000100000011101110110100101110100011010000010000001101101011101010110110001110100011010010111000001101100011001010010000001110101011100110110010101110010001000000110010001100101011101100110100101100011011001010111001100100000011100110111010101100011011010000010000001100001011100110010000001101101011011110110001001101001011011000110010100100000011100000110

In [39]:
finalBytesInString=byteToStr[:11160] #TRUNCATING STRING W TO 11336 SO THAT IT IS DIVISIBLE BY 8 FOR DECODING

In [35]:
#WE CONVERT OUR BYTESTREAM IN STRING TO BINARY NUM AND THEN TO BYTES ARRAY, 
#THE ARRAY THUS FORMULATED IS THEN CONVERTED INTO ASCII TEXT WHICH IS THEN DECODED INTO ASCII VALUE
 
print((int(finalBytesInString,2).to_bytes(int(z,2).bit_length(), "big")).decode())

#print(int(finalBytesInString,2))

#print((int(finalBytesInString,2).to_bytes(int(z,2).bit_length(),"big")))


NameError: name 'z' is not defined

In [40]:
len(finalBytesInString)

11160

In [41]:
z=finalBytesInString
binary_int = int(z,2);
 
# Getting the byte number
byte_number = binary_int.bit_length() + 7 // 8
 
# Getting an array of bytes
binary_array = binary_int.to_bytes(byte_number, "big")
 
# Converting the array into ASCII text
ascii_text = binary_array.decode()
 
# Getting the ASCII value
print(ascii_text)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        